In [31]:
import pandas as pd
import numpy as np
import re
import pickle
from tqdm import tqdm



from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


# Adding the parent directory to the path so that the `src` directory can be imported.
import sys
sys.path.append("../")

# Importing the `support` module from the `src` directory.
import src.support as sp

# Ignoring the warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
# abrimos el pickle de los actores más representado
with open('../data/pickle/top10_actors.pickle', 'rb') as act:
    actors = pickle.load(act)

In [3]:
actors

['Adam Sandler',
 'Maya Rudolph',
 'Andrew Bachelor',
 'Lakeith Stanfield',
 'Jacki Weaver',
 'Ken Marino',
 'Robbie Amell',
 'Zachary Quinto',
 'Nick Swardson',
 'Rob Schneider']

In [40]:
def scrap_imdb(list_): 

    personal_actors = {"name": [], 
                    "image": [], 
                    "description": [], 
                    "positions": [], 
                    "known_for": [], 
                    "creditos": []}

    
    eleccion = input("Que quieres escrapear, director o actor:")
    
    while eleccion.lower() not in ["director", "actor"]:
        print("Lo sentimos elige una opción correcta entre director y actor")
        eleccion = input("Que quieres escrapear, director o actor:")

        
    
    for element in tqdm(list_): 
        print(element)
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.implicitly_wait(10)

        driver.get("https://www.imdb.com/")
        # metemos el actor
        driver.find_element("css selector", "#suggestion-search").send_keys(element, Keys.ENTER)
        driver.implicitly_wait(10)
        
        
        # hacemos click en la actriz o actor
        driver.find_element("css selector", '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-5c7dd5e4-0.iRUEko > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.fgqnfI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c').click()

        if eleccion == "director": 
            pass
        else:
            #capturamos los creditos
            personal_actors["creditos"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.hWwhTB > div > section > div > div.sc-4253d404-1.hwVyrY.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.ipc-chip-list--base.ipc-chip-list.ipc-chip-list--nowrap.sc-2919cd96-0.hNbCTu.nm_flmg_chips > div.ipc-chip-list__scroller > div.filmography-chip-group.ipc-chip-group").text)

        
        personal_actors["name"].append(element)
        try:
            personal_actors["positions"].append(driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.hWwhTB > section > div:nth-child(4) > section > section > div.sc-d3e8adc5-3.kIpvxT > div > ul').text)
        except: 
             personal_actors["positions"].append(np.nan)
                
                
        # capturamos conocido por
        try:
            personal_actors["known_for"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.hWwhTB > div > section > div > div.sc-4253d404-1.hwVyrY.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-5eb32b71-0.jVJjba > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid").text)
        except:
            personal_actors["known_for"].append(np.nan)
            
        
        driver.implicitly_wait(30)
        # entro en la biografía
        driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.hWwhTB > section > div:nth-child(4) > section > section > div.sc-383649c6-0.fOBDDX > div > div:nth-child(2) > ul > li:nth-child(1) > a").click()
        
        # capturamos la descripcion
        try:
            personal_actors["description"].append(driver.find_element("css selector", '#bio_content > div:nth-child(9)').text)
        except:
            personal_actors["description"].append(np.nan)
            
        # capturamos la imagen
        try:
            personal_actors["image"].append(driver.find_element("css selector", '#main > div.article.listo > div.subpage_title_block.name-subpage-header-block > a > img').get_attribute("src"))
        except:
            personal_actors["image"].append(np.nan)
            
        
    driver.quit()
    if eleccion == "director":
        personal_actors.pop("creditos")
        return pd.DataFrame(personal_actors)
    elif eleccion == "actor":
        return pd.DataFrame(personal_actors)

In [5]:
# scrapping información personal de cada actor
actors_df = scrap_imdb(actors)

Que quieres escrapear, director o actor:actor


In [6]:
actors_df.head(2)

,name,image,description,positions,known_for,creditos
0,Adam Sandler,https://m.media-amazon.com/images/M/MV5BMjQyNz...,"Adam Richard Sandler was born September 9, 196...",Reparto\nProducción\nGuion,"El aguador\n6,1\nBobby Boucher\n1998\nEl chico...",Banda sonora34\nGuion y continuidad7\nComposic...
1,Maya Rudolph,https://m.media-amazon.com/images/M/MV5BMTMzOT...,"Maya Rudolph was born on July 27, 1972 in Gain...",Reparto\nProducción\nGuion,"La boda de mi mejor amiga\n6,8\nLillian\n2011\...",Banda sonora29\nMúsica1\nComo sí mismos120\nAg...


In [7]:
actors_df["positions"] = actors_df["positions"].str.replace("\n", " ")

In [8]:
actors_df.head(1)

,name,image,description,positions,known_for,creditos
0,Adam Sandler,https://m.media-amazon.com/images/M/MV5BMjQyNz...,"Adam Richard Sandler was born September 9, 196...",Reparto Producción Guion,"El aguador\n6,1\nBobby Boucher\n1998\nEl chico...",Banda sonora34\nGuion y continuidad7\nComposic...


In [9]:
actors_df["creditos"] = actors_df["creditos"].str.split("\n")

In [10]:
for indice, fila in actors_df.iterrows():
    for valores in fila["creditos"]:
        nombre_col = re.findall("[^0-9]+", valores)[0]
        
        if nombre_col not in actors_df.columns:
            actors_df[nombre_col] = np.nan
        else:
            pass
        actors_df.loc[indice, nombre_col] = int(re.findall("[0-9]+", valores)[0])
        
        

In [11]:
actors_df.head(1)

,name,image,description,positions,known_for,creditos,Banda sonora,Guion y continuidad,Composición,sonido de archivo,Música,Como sí mismos,Agradecimientos,metraje de archivo,Producción,Dirección,Dirección o asistencia de la segunda unidad,Equipos adicionales
0,Adam Sandler,https://m.media-amazon.com/images/M/MV5BMjQyNz...,"Adam Richard Sandler was born September 9, 196...",Reparto Producción Guion,"El aguador\n6,1\nBobby Boucher\n1998\nEl chico...","[Banda sonora34, Guion y continuidad7, Composi...",34.0,7.0,3.0,2.0,1.0,256.0,10.0,80.0,NaN,NaN,NaN,NaN


In [16]:
actors_df.drop(["creditos"], axis = 1, inplace = True)

In [12]:
actors_df["known_for"] = actors_df["known_for"].str.split("\n")

In [17]:
actors_df = actors_df.explode("known_for")

In [20]:
actors_df.head(2)

,name,image,description,positions,known_for,Banda sonora,Guion y continuidad,Composición,sonido de archivo,Música,Como sí mismos,Agradecimientos,metraje de archivo,Producción,Dirección,Dirección o asistencia de la segunda unidad,Equipos adicionales
0,Adam Sandler,https://m.media-amazon.com/images/M/MV5BMjQyNz...,"Adam Richard Sandler was born September 9, 196...",Reparto Producción Guion,El aguador,34.0,7.0,3.0,2.0,1.0,256.0,10.0,80.0,NaN,NaN,NaN,NaN
0,Adam Sandler,https://m.media-amazon.com/images/M/MV5BMjQyNz...,"Adam Richard Sandler was born September 9, 196...",Reparto Producción Guion,"6,1",34.0,7.0,3.0,2.0,1.0,256.0,10.0,80.0,NaN,NaN,NaN,NaN


In [21]:
actors_df.to_csv("../data/top10_actors.csv", index = False, sep = ";")


# Directors

In [25]:
# open the top10 directors pickle file
with open('../data/pickle/top10_directors.pickle', 'rb') as dir:
    directors = pickle.load(dir)
directors

['McG',
 'Blair Simmons',
 'John Schultz',
 'Noah Baumbach',
 'Karan Johar',
 'Dibakar Banerjee',
 'Julien Leclercq',
 'Amy Poehler',
 'Kyle Newacheck',
 'Peter Sullivan']

In [41]:
# scrapping información personal directores 
directors_df  = scrap_imdb(directors)

Que quieres escrapear, director o actor:director


  0%|                                                    | 0/10 [00:00<?, ?it/s]

McG


 10%|████▍                                       | 1/10 [00:12<01:51, 12.35s/it]

Blair Simmons


...

 80%|███████████████████████████████████▏        | 8/10 [03:44<00:57, 28.95s/it]

Kyle Newacheck


 90%|███████████████████████████████████████▌    | 9/10 [03:58<00:24, 24.28s/it]

Peter Sullivan


100%|███████████████████████████████████████████| 10/10 [04:42<00:00, 28.28s/it]


In [42]:
directors_df.head()

,name,image,description,positions,known_for
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción\nEquipos adicionales\nDirección,"Los ángeles de Charlie: Al límite\n4,9\nDirecc..."
1,Blair Simmons,NaN,NaN,Dirección\nAnimación\nArte,"Blaze y los Monster Machines\n5,7\nSerie de TV..."
2,John Schultz,https://m.media-amazon.com/images/M/MV5BMTY5MD...,Schultz was an original member of the Charlott...,Dirección\nGuion\nProducción,"Bandwagon\n7,1\nDirección\n1996\nUna pandilla ..."
3,Noah Baumbach,https://m.media-amazon.com/images/M/MV5BMjA3NT...,Born in Brooklyn in 1969 Noah Baumbach is the ...,Guion\nDirección\nProducción,"Historia de un matrimonio\n7,9\nGuion\n2019\nU..."
4,Karan Johar,https://m.media-amazon.com/images/M/MV5BMTM5Nj...,"A close friend of Shahrukh Khan, he assisted Y...",Producción\nReparto\nGuion,"Algo sucede en mi corazón\n7,6\nGuion\n1998\nN..."


In [43]:
directors_df["positions"] = directors_df["positions"].str.replace("\n", " ")
directors_df["known_for"] = directors_df["known_for"].str.split("\n")

In [44]:
directors_df = directors_df.explode("known_for")

In [45]:
directors_df.head()

,name,image,description,positions,known_for
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción Equipos adicionales Dirección,Los ángeles de Charlie: Al límite
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción Equipos adicionales Dirección,"4,9"
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción Equipos adicionales Dirección,Dirección
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción Equipos adicionales Dirección,2003
0,McG,https://m.media-amazon.com/images/M/MV5BMTI1MT...,"McG was born on August 9, 1968 in Kalamazoo, M...",Producción Equipos adicionales Dirección,Terminator: Salvation


In [46]:
directors_df.to_csv("../data/top10_directors.csv", index=False, sep= ";")
